In [1]:
# -*- coding: utf-8 -*-
import json
from rich.console import Console
from rich.table import Table
import math
import sys
from collections import defaultdict

# --- Configuration ---
FILENAME = "bce_C0_beta_0.json"  # 在這裡定義檔案名稱
console = Console(highlight=False)  # 關閉自動高亮以使用自訂色彩
METRICS_ORDER = [
    "overall_mAP", "overall_f1", "head_mAP", "medium_mAP", "tail_mAP",
    "head_f1", "medium_f1", "tail_f1", "overall_acc"
]
METRIC_FRIENDLY_NAMES = {
    "overall_mAP": "Overall mAP", "overall_f1": "Overall F1",
    "head_mAP": "Head mAP", "medium_mAP": "Medium mAP", "tail_mAP": "Tail mAP",
    "head_f1": "Head F1", "medium_f1": "Medium F1", "tail_f1": "Tail F1",
    "overall_acc": "Overall Acc"
}
DATA_TYPES = ["crop", "part", "symptomCategories", "symptomTags"]
PRECISION = 2
# 設定名稱列的最大寬度，現在顯示 fusion 模式，可能不需要太寬
SETTING_NAME_MAX_WIDTH = 20 # 可以根據需要調整

# --- 深色主題顏色配置 ---
COLORS = {
    "title": "bright_cyan",
    "data_type": "bright_green",
    "table_title": "bright_magenta",
    "setting_header": "bright_cyan",   # 行標題（設定名稱）的樣式
    "metric_header": "bright_yellow",  # 列標題（指標名稱）的樣式
    "mean_value": "bright_white",      # 平均值的顏色
    "std_dev": "dim bright_white",     # 標準差的顏色 (dim 表示稍暗)
    "error": "bright_red",
    "warning": "bright_yellow",
    "separator": "bright_white",
    "na": "gray50",
}

# --- Helper Functions ---
def format_value(value, is_percent=True, precision=PRECISION, for_std_dev=False):
    """格式化數值，處理 None 和 NaN。"""
    if value is None or (isinstance(value, float) and math.isnan(value)):
        return f"[{COLORS['na']}]N/A[/{COLORS['na']}]"
    try:
        val_float = float(value)
        if for_std_dev and abs(val_float) < 1e-9 and abs(val_float) != 0.0:
             return f"{val_float:.{precision}e}"
        elif is_percent:
            return f"{val_float:.{precision}f}%"
        else:
            return f"{val_float:.{precision}f}"
    except (ValueError, TypeError):
        return f"[{COLORS['na']}]N/A[/{COLORS['na']}]"

def format_mean_std(mean, std, is_percent, precision=PRECISION):
    """將平均值和標準差格式化為單個字串"""
    mean_str = format_value(mean, is_percent=is_percent, precision=precision)
    if std is None or (isinstance(std, float) and math.isnan(std)) or abs(float(std)) < 1e-9:
        return mean_str
    else:
        std_str = format_value(std, is_percent=False, precision=precision, for_std_dev=True)
        return f"[{COLORS['mean_value']}]{mean_str}[/{COLORS['mean_value']}] [{COLORS['std_dev']}]±{std_str}[/{COLORS['std_dev']}]"

def extract_fusion_mode(setting_name):
    """從完整的設定名稱中提取 Fusion 模式"""
    fusion_prefix = "Fusion="
    fusion_start_index = setting_name.find(fusion_prefix)

    if fusion_start_index != -1:
        start = fusion_start_index + len(fusion_prefix)
        next_underscore_index = setting_name.find("_", start)
        if next_underscore_index != -1:
            return setting_name[start:next_underscore_index]
        else:
            return setting_name[start:] # 如果後面沒有 _，則取到結尾
    return setting_name # 如果找不到 "Fusion="，返回原始名稱

# --- Load Data from File ---
try:
    with open(FILENAME, 'r', encoding='utf-8') as f:
        data = json.load(f)
except FileNotFoundError:
    console.print(f"[bold {COLORS['error']}]錯誤: 找不到檔案:[/bold {COLORS['error']}] '{FILENAME}'")
    sys.exit(1)
except json.JSONDecodeError:
    console.print(f"[bold {COLORS['error']}]錯誤: 檔案中的 JSON 格式無效:[/bold {COLORS['error']}] '{FILENAME}'")
    sys.exit(1)

# --- Processing ---
if not data or not isinstance(data, dict):
    console.print(f"[bold {COLORS['error']}]錯誤: 載入的 JSON 數據為空或不是預期的字典格式。[/bold {COLORS['error']}]")
    sys.exit(1)

# --- 1. 收集所有設定的數據 (包含 std) ---
comparison_data = defaultdict(lambda: defaultdict(dict))
setting_names = list(data.keys())

for setting_name, results_data in data.items():
    if not isinstance(results_data, dict):
        console.print(f"[{COLORS['warning']}]跳過項目 '{setting_name}'，因為它不是字典。[/{COLORS['warning']}]")
        continue
    avg_metrics_data = results_data.get("avg_metrics", {})
    for data_type in DATA_TYPES:
        data_type_avg_metrics = avg_metrics_data.get(data_type, {})
        for metric_key in METRICS_ORDER:
            metric_data = data_type_avg_metrics.get(metric_key)
            mean_value = None
            std_value = None
            if isinstance(metric_data, dict):
                 mean_value = metric_data.get("mean")
                 std_value = metric_data.get("std")
            comparison_data[data_type][setting_name][metric_key] = {
                "mean": mean_value,
                "std": std_value
            }

# --- 2. 為每個 data_type 生成轉置的比較表 ---
console.print(f"\n[bold {COLORS['title']}]跨設定平均指標比較 (含標準差，行列互換)[/bold {COLORS['title']}]")

for data_type in DATA_TYPES:
    has_any_data = False
    for setting_name in setting_names:
        if data_type in comparison_data and setting_name in comparison_data[data_type]:
            if any(metric_key in comparison_data[data_type][setting_name] and
                   comparison_data[data_type][setting_name][metric_key].get("mean") is not None
                   for metric_key in METRICS_ORDER):
                 has_any_data = True
                 break
    if not has_any_data:
        console.print(f"\n[{COLORS['warning']}]未找到 '{data_type}' 數據類型的任何平均指標數據，跳過此類型。[/{COLORS['warning']}]")
        continue

    console.print(f"\n[bold {COLORS['data_type']}]--- {data_type.capitalize()} 數據類型 ---[/bold {COLORS['data_type']}]")

    comparison_table = Table(
        title=f"跨設定 {data_type.capitalize()} 平均指標比較 (Mean ± Std)",
        show_header=True,
        header_style=f"bold {COLORS['table_title']}",
        border_style=COLORS["separator"],
        title_style=f"bold {COLORS['title']}",
        expand=True
    )

    # 添加第一列：設定名稱 (Fusion Mode)
    comparison_table.add_column("Fusion Mode", style=f"bold {COLORS['setting_header']}", min_width=10, max_width=SETTING_NAME_MAX_WIDTH, overflow="fold")

    # 動態添加每個指標的列
    metric_columns_added = []
    for metric_key in METRICS_ORDER:
         if any(setting_name in comparison_data[data_type] and
                metric_key in comparison_data[data_type][setting_name] and
                comparison_data[data_type][setting_name][metric_key].get("mean") is not None
                for setting_name in setting_names):
            friendly_name = METRIC_FRIENDLY_NAMES.get(metric_key, metric_key)
            comparison_table.add_column(friendly_name, justify="right", header_style=f"bold {COLORS['metric_header']}")
            metric_columns_added.append(metric_key)

    # 填充表格行 (每個設定為一行)
    has_rows = False
    for setting_name in setting_names:
        # 檢查此設定在此 data_type 下是否有任何數據
        if not any(metric_key in comparison_data[data_type].get(setting_name, {}) and
                   comparison_data[data_type][setting_name][metric_key].get("mean") is not None
                   for metric_key in metric_columns_added):
            continue

        # *** 修改：提取 Fusion 模式作為顯示名稱 ***
        display_name = extract_fusion_mode(setting_name)
        # 如果提取後為空，則回退到原始名稱（以防萬一）
        if not display_name:
            display_name = setting_name

        row_data = [display_name] # 第一個元素是提取後的 Fusion Mode

        # 按照添加的指標列順序填充數據
        for metric_key in metric_columns_added:
            metric_values = comparison_data[data_type][setting_name].get(metric_key, {"mean": None, "std": None})
            mean_val = metric_values.get("mean")
            std_val = metric_values.get("std")

            is_percent = any(sub in metric_key.lower() for sub in ['map', 'f1', 'acc'])
            formatted_cell = format_mean_std(mean_val, std_val, is_percent, PRECISION)
            row_data.append(formatted_cell)

        comparison_table.add_row(*row_data)
        has_rows = True

    # 僅當表格有行時才打印
    if has_rows:
        console.print(comparison_table)
    else:
        console.print(f"[{COLORS['warning']}]未能為 '{data_type}' 生成任何比較行。[/{COLORS['warning']}]")

    console.print(f"[{COLORS['separator']}]" + "=" * 80 + f"[/{COLORS['separator']}]")

# --- End of Script ---

跨設定平均指標比較 (含標準差，行列互換)

--- Crop 數據類型 ---

                                       跨設定 Crop 平均指標比較 (Mean ± Std)                                       
┏━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━
┃ Fusion     ┃   Overall ┃   Overall ┃          ┃    Medium ┃          ┃         ┃           ┃         ┃   Overall 
┃ Mode       ┃       mAP ┃        F1 ┃ Head mAP ┃       mAP ┃ Tail mAP ┃ Head F1 ┃ Medium F1 ┃ Tail F1 ┃       Acc 
┡━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━
│ film       │   100.00% │   100.00% │  100.00% │   100.00% │  100.00% │ 100.00% │   100.00% │ 100.00% │   100.00% 
└────────────┴───────────┴───────────┴──────────┴───────────┴──────────┴─────────┴───────────┴─────────┴───────────

================================================================================

--- Part 數據類型 ---

...

                                 跨設定 Symptomcategories 平均指標比較 (Mean ± Std)                                
┏━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━
┃ Fusion     ┃  Overall ┃   Overall ┃          ┃    Medium ┃          ┃           ┃   Medium ┃           ┃  Overall
┃ Mode       ┃      mAP ┃        F1 ┃ Head mAP ┃       mAP ┃ Tail mAP ┃   Head F1 ┃       F1 ┃   Tail F1 ┃      Acc
┡━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━
│ film       │   84.91% │    76.67% │   94.91% │    91.07% │   67.21% │    88.13% │   85.69% │    53.92% │   95.85%
│            │    ±0.43 │     ±1.89 │    ±0.05 │     ±0.36 │    ±1.61 │     ±0.17 │    ±0.39 │     ±5.59 │    ±0.04
└────────────┴──────────┴───────────┴──────────┴───────────┴──────────┴───────────┴──────────┴───────────┴─────────

================================================================================

--- Symptomtags 數據類型 ---

                                    跨設定 Symptomtags 平均指標比較 (Mean ± Std)                                   
┏━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━
┃ Fusion     ┃  Overall ┃   Overall ┃          ┃    Medium ┃          ┃           ┃   Medium ┃           ┃  Overall
┃ Mode       ┃      mAP ┃        F1 ┃ Head mAP ┃       mAP ┃ Tail mAP ┃   Head F1 ┃       F1 ┃   Tail F1 ┃      Acc
┡━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━
│ film       │   66.21% │    52.79% │   84.17% │    74.36% │   22.87% │    77.67% │   56.27% │     4.17% │   97.84%
│            │    ±0.76 │     ±1.67 │    ±0.07 │     ±1.56 │    ±3.66 │     ±0.28 │    ±2.37 │     ±4.25 │    ±0.02
└────────────┴──────────┴───────────┴──────────┴───────────┴──────────┴───────────┴──────────┴───────────┴─────────

================================================================================